# Cleaning up BBR data

The data imported from BBR includes a lot of obviously wrong records, such as buildings with negative floor area or a negative number of floors. Before going further, we need to clean up the dataset.

### Setup

In [ ]:
import psycopg as pg
import pandas as pd
from IPython.display import clear_output # Function to clear output when counting rows

In [ ]:
with open('database_parameters.txt','r') as f: # Text file containing parameters to connect to the database
    params=f.read()
    f.close()

In [ ]:
def run_sql (DBparameters,SQLcode):
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(DBparameters)

        # create a new cursor
        cur = connector.cursor()

        # execute the SQL statement
        cur.execute(SQLcode)

        # commit the changes to the database
        connector.commit()

        # close communication with the database
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(error)

    finally:
        if connector is not None:
            connector.close()

Let's insert a new table in the database to store copies of the records we might modify or delete.

In [ ]:
SQLdeleted="""
CREATE TABLE IF NOT EXISTS public.deleted_buildings
(
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    jordstykke integer,
    byg007Bygningsnummer integer,
    grund character varying(50) COLLATE pg_catalog."default",
    husnummer character varying(50) COLLATE pg_catalog."default",
    byg404Koordinat character varying(40) COLLATE pg_catalog."default",
    byg026Opførelsesår smallint,
    byg027OmTilbygningsår smallint,
    byg021BygningensAnvendelse character varying(30) COLLATE pg_catalog."default",
    byg041BebyggetAreal integer,
    byg038SamletBygningsareal integer,
    byg040BygningensSamledeErhvervsAreal integer,
    byg039BygningensSamledeBoligAreal integer,
    byg042ArealIndbyggetGarage integer,
    byg043ArealIndbyggetCarport integer,
    byg044ArealIndbyggetUdhus integer,
    byg045ArealIndbyggetUdestueEllerLign integer,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen integer,
    byg047ArealAfAffaldsrumITerrænniveau integer,
    byg048AndetAreal integer,
    byg049ArealAfOverdækketAreal integer,
    byg050ArealÅbneOverdækningerPåBygningenSamlet integer,
    byg051Adgangsareal integer,
    byg054AntalEtager smallint,
    byg055AfvigendeEtager smallint,
    byg056Varmeinstallation smallint,
    byg057Opvarmningsmiddel smallint,
    byg058SupplerendeVarme smallint,
    byg032YdervæggensMateriale smallint,
    byg034SupplerendeYdervæggensMateriale smallint,
    byg033Tagdækningsmateriale smallint,
    byg035SupplerendeTagdækningsMateriale smallint,
    byg036AsbestholdigtMateriale smallint,
    byg130ArealAfUdvendigEfterisolering integer,
    byg071BevaringsværdighedReference smallint,
    byg150Gulvbelægning character varying(50) COLLATE pg_catalog."default",
    byg151Frihøjde character varying(50) COLLATE pg_catalog."default",
    roof_pitch smallint,
    int_wall_surface_lb real,
    int_wall_surface_nlb real,
    CONSTRAINT deleted_buildings_pkey PRIMARY KEY (id_lokalId)
);
"""

In [ ]:
run_sql(params,SQLdeleted)

In the following steps, we will first identify problematic building records and copy them into separate tables (to keep track of the original data). Then, we will delete them from the original record.

In [ ]:
SQLcopy="""
INSERT INTO deleted_buildings 
(
    id_lokalId,
    kommunekode,
    jordstykke,
    byg007Bygningsnummer,
    grund,
    husnummer,
    byg404Koordinat,
    byg026Opførelsesår,
    byg027OmTilbygningsår,
    byg021BygningensAnvendelse,
    byg041BebyggetAreal,
    byg038SamletBygningsareal,
    byg040BygningensSamledeErhvervsAreal,
    byg039BygningensSamledeBoligAreal,
    byg042ArealIndbyggetGarage,
    byg043ArealIndbyggetCarport,
    byg044ArealIndbyggetUdhus,
    byg045ArealIndbyggetUdestueEllerLign,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen,
    byg047ArealAfAffaldsrumITerrænniveau,
    byg048AndetAreal,
    byg049ArealAfOverdækketAreal,
    byg050ArealÅbneOverdækningerPåBygningenSamlet,
    byg051Adgangsareal,
    byg054AntalEtager,
    byg055AfvigendeEtager,
    byg056Varmeinstallation,
    byg057Opvarmningsmiddel,
    byg058SupplerendeVarme,
    byg032YdervæggensMateriale,
    byg034SupplerendeYdervæggensMateriale,
    byg033Tagdækningsmateriale,
    byg035SupplerendeTagdækningsMateriale,
    byg036AsbestholdigtMateriale,
    byg130ArealAfUdvendigEfterisolering,
    byg071BevaringsværdighedReference,
    byg150Gulvbelægning,
    byg151Frihøjde,
    roof_pitch,
    int_wall_surface_lb,
    int_wall_surface_nlb
)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
ON CONFLICT ON CONSTRAINT deleted_buildings_pkey DO NOTHING
"""

### Finding buildings with negative floor areas or number of floors

In [ ]:
conn = None
try:
    conn = pg.connect(params, row_factory=pg.rows.dict_row)
    cur = conn.cursor()
    #cur.execute("DELETE FROM deleted_buildings")
    cur.execute("SELECT * FROM buildings ORDER BY id_lokalId")
    row = cur.fetchone() # Get properties from the first building as a dictionary
    print(row)
    
    row_number=0

    while row is not None:
        row_number+=1
        clear_output(wait=True)
        bbrid=row['id_lokalId'.lower()]
        builtarea=row['byg041BebyggetAreal'.lower()]
        floorarea=row['byg038SamletBygningsareal'.lower()]
        nfloors=row['byg054AntalEtager'.lower()]
        print(row_number,bbrid,builtarea,floorarea,nfloors)
        if ((builtarea is None or builtarea == 0) and (floorarea is None or floorarea == 0)) or (builtarea is not None and builtarea < 0) or (floorarea is not None and floorarea < 0) or (nfloors is not None and nfloors<0):
            cur_copy=conn.cursor()
            cur_copy.execute(SQLcopy, tuple(row.values()))
            conn.commit()
            cur_copy.close()
        
        row=cur.fetchone() # Retrieve the next building as a tuple and iterate
    
    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

### Finding buildings with no construction year

In [ ]:
conn = None
try:
    conn = pg.connect(params, row_factory=pg.rows.dict_row)
    cur = conn.cursor()
    #cur.execute("DELETE FROM deleted_buildings")
    cur.execute("SELECT * FROM buildings WHERE byg026Opførelsesår IS NULL ORDER BY id_lokalId")
    row = cur.fetchone() # Get properties from the first building as a dictionary
    print(row)
    
    row_number=0

    while row is not None:
        row_number+=1
        clear_output(wait=True)
        print(row_number, row)
        cur_copy=conn.cursor()
        cur_copy.execute(SQLcopy, tuple(row.values()))
        conn.commit()
        cur_copy.close()
        
        row=cur.fetchone() # Retrieve the next building as a tuple and iterate
    
    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

### Finding buildings with no suitable wall or roof material


In [ ]:
conn = None
try:
    conn = pg.connect(params, row_factory=pg.rows.dict_row)
    cur = conn.cursor()
    #cur.execute("DELETE FROM deleted_buildings")
    cur.execute("SELECT * FROM buildings WHERE (byg032ydervæggensmateriale IS NULL OR byg032ydervæggensmateriale IN ('80','90','11') OR byg033tagdækningsmateriale IS NULL) ORDER BY id_lokalId")
    row = cur.fetchone() # Get properties from the first building as a dictionary
    print(row)
    
    row_number=0

    while row is not None:
        row_number+=1
        clear_output(wait=True)
        print(row_number, row)
        cur_copy=conn.cursor()
        cur_copy.execute(SQLcopy, tuple(row.values()))
        conn.commit()
        cur_copy.close()
        
        row=cur.fetchone() # Retrieve the next building as a tuple and iterate
    
    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

### Finding problematic outliers

First, let's compute some descriptive statistics for our data:

In [ ]:
conn = None
try:
    conn = pg.connect(params, row_factory=pg.rows.dict_row)
    cur = conn.cursor()
    cur.execute("SELECT AVG(byg041BebyggetAreal) AVG_footprint, STDDEV_POP(byg041BebyggetAreal) STD_footprint, AVG(byg038SamletBygningsareal) AVG_area, STDDEV_POP(byg038SamletBygningsareal) STD_area, AVG(byg054AntalEtager) AVG_floors, STDDEV_POP(byg054AntalEtager) STD_floors FROM buildings")
    row = cur.fetchone() # Get properties from the first building as a dictionary
    print(row)
    
    AVG_footprint=row['AVG_footprint'.lower()]
    STD_footprint=row['STD_footprint'.lower()]
    AVG_area=row['AVG_area'.lower()]
    STD_area=row['STD_area'.lower()]
    AVG_floors=row['AVG_floors'.lower()]
    STD_floors=row['STD_floors'.lower()]

    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

In [ ]:
conn = None
try:
    conn = pg.connect(params, row_factory=pg.rows.dict_row)
    cur = conn.cursor()
    SQL="""SELECT 
    PERCENTILE_CONT(0.1) WITHIN GROUP (ORDER BY byg041BebyggetAreal) q10_footprint,
    PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY byg041BebyggetAreal) q90_footprint,
    PERCENTILE_CONT(0.1) WITHIN GROUP (ORDER BY byg038SamletBygningsareal) q10_area,
    PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY byg038SamletBygningsareal) q90_area,
    PERCENTILE_CONT(0.1) WITHIN GROUP (ORDER BY byg054AntalEtager) q10_floors,
    PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY byg054AntalEtager) q90_floors
    FROM buildings
    """
    cur.execute(SQL)
    row = cur.fetchone() # Get properties from the first building as a dictionary
    print(row)
    
    q10_footprint=row['q10_footprint'.lower()]
    q90_footprint=row['q90_footprint'.lower()]
    q10_area=row['q10_area'.lower()]
    q90_area=row['q90_area'.lower()]
    q10_floors=row['q10_floors'.lower()]
    q90_floors=row['q90_floors'.lower()]

    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

The percentiles are not very useful here - even the 1% and 99% percentiles still give reasonable values.

Outliers based on footprint and floor areas do not necessarily represent wrong data - there are some very large buildings, but they are warehouses or agricultural facilities, so they might represent actual large buildings. However, some outliers for number of floors are clearly wrong. We can delete buildings where the number of floors is equal to the footprint or the floor area (a mistake in data entry), or the number of floors is higher than 30 (the three tallest skyscrapers in Denmark have 25, 30 and 25 floors respectively)

In [ ]:
SQLoutliers="""
CREATE TABLE IF NOT EXISTS public.outlier_buildings
(
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    jordstykke integer,
    byg007Bygningsnummer integer,
    grund character varying(50) COLLATE pg_catalog."default",
    husnummer character varying(50) COLLATE pg_catalog."default",
    byg404Koordinat character varying(40) COLLATE pg_catalog."default",
    byg026Opførelsesår smallint,
    byg027OmTilbygningsår smallint,
    byg021BygningensAnvendelse character varying(30) COLLATE pg_catalog."default",
    byg041BebyggetAreal integer,
    byg038SamletBygningsareal integer,
    byg040BygningensSamledeErhvervsAreal integer,
    byg039BygningensSamledeBoligAreal integer,
    byg042ArealIndbyggetGarage integer,
    byg043ArealIndbyggetCarport integer,
    byg044ArealIndbyggetUdhus integer,
    byg045ArealIndbyggetUdestueEllerLign integer,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen integer,
    byg047ArealAfAffaldsrumITerrænniveau integer,
    byg048AndetAreal integer,
    byg049ArealAfOverdækketAreal integer,
    byg050ArealÅbneOverdækningerPåBygningenSamlet integer,
    byg051Adgangsareal integer,
    byg054AntalEtager smallint,
    byg055AfvigendeEtager smallint,
    byg056Varmeinstallation smallint,
    byg057Opvarmningsmiddel smallint,
    byg058SupplerendeVarme smallint,
    byg032YdervæggensMateriale smallint,
    byg034SupplerendeYdervæggensMateriale smallint,
    byg033Tagdækningsmateriale smallint,
    byg035SupplerendeTagdækningsMateriale smallint,
    byg036AsbestholdigtMateriale smallint,
    byg130ArealAfUdvendigEfterisolering integer,
    byg071BevaringsværdighedReference smallint,
    byg150Gulvbelægning character varying(50) COLLATE pg_catalog."default",
    byg151Frihøjde character varying(50) COLLATE pg_catalog."default",
    roof_pitch smallint,
    int_wall_surface_lb real,
    int_wall_surface_nlb real,
    CONSTRAINT outlier_buildings_pkey PRIMARY KEY (id_lokalId)
);
"""

In [ ]:
run_sql(params,SQLoutliers)

In [ ]:
SQLcopyoutliers="""
INSERT INTO outlier_buildings 
(
    id_lokalId,
    kommunekode,
    jordstykke,
    byg007Bygningsnummer,
    grund,
    husnummer,
    byg404Koordinat,
    byg026Opførelsesår,
    byg027OmTilbygningsår,
    byg021BygningensAnvendelse,
    byg041BebyggetAreal,
    byg038SamletBygningsareal,
    byg040BygningensSamledeErhvervsAreal,
    byg039BygningensSamledeBoligAreal,
    byg042ArealIndbyggetGarage,
    byg043ArealIndbyggetCarport,
    byg044ArealIndbyggetUdhus,
    byg045ArealIndbyggetUdestueEllerLign,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen,
    byg047ArealAfAffaldsrumITerrænniveau,
    byg048AndetAreal,
    byg049ArealAfOverdækketAreal,
    byg050ArealÅbneOverdækningerPåBygningenSamlet,
    byg051Adgangsareal,
    byg054AntalEtager,
    byg055AfvigendeEtager,
    byg056Varmeinstallation,
    byg057Opvarmningsmiddel,
    byg058SupplerendeVarme,
    byg032YdervæggensMateriale,
    byg034SupplerendeYdervæggensMateriale,
    byg033Tagdækningsmateriale,
    byg035SupplerendeTagdækningsMateriale,
    byg036AsbestholdigtMateriale,
    byg130ArealAfUdvendigEfterisolering,
    byg071BevaringsværdighedReference,
    byg150Gulvbelægning,
    byg151Frihøjde,
    roof_pitch,
    int_wall_surface_lb,
    int_wall_surface_nlb
)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
ON CONFLICT ON CONSTRAINT outlier_buildings_pkey DO NOTHING"""

In [ ]:
conn = None
try:
    conn = pg.connect(params, row_factory=pg.rows.dict_row)
    cur = conn.cursor()
    cur.execute("DELETE FROM modified_buildings")
    cur.execute("SELECT * FROM buildings ORDER BY id_lokalId")
    row = cur.fetchone() # Get properties from the first building as a dictionary
    
    row_number=0

    while row is not None:
        row_number+=1
        clear_output(wait=True)
        bbrid=row['id_lokalId'.lower()]
        builtarea=row['byg041BebyggetAreal'.lower()]
        floorarea=row['byg038SamletBygningsareal'.lower()]
        nfloors=row['byg054AntalEtager'.lower()]
        print(row_number,bbrid,builtarea,floorarea,nfloors)
        if nfloors == builtarea or nfloors == floorarea or (nfloors is not None and nfloors > 30):
            cur_copy=conn.cursor()
            cur_copy.execute(SQLcopyoutliers, tuple(row.values()))
            conn.commit()
            cur_copy.close()
        
        row=cur.fetchone() # Retrieve the next building as a tuple and iterate
    
    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

In [ ]:
conn = None
try:
    conn = pg.connect(params)
    cur = conn.cursor()
    cur.execute("SELECT id_lokalId FROM outlier_buildings ORDER BY id_lokalId")
    outlier_buildings = cur.fetchall() 
    conn.commit()
    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

In [ ]:
delete_values(outlier_buildings,last_id_deleted='')

### Deleting copied buildings


Now let's delete the previously identified buildings from the original table.

In [ ]:
def delete_building(bbrid):
    conn = None
    try:
        conn = pg.connect(params)
        cur = conn.cursor()
        cur.execute("DELETE FROM buildings WHERE id_lokalId = %s", (bbrid,))
        conn.commit()
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print('error:', error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
def delete_values(deleted_buildings,last_id_deleted=''):
    isNewBuilding=False 
    if last_id_deleted=='':
        isNewBuilding=True 
    count=0
    
    for row in deleted_buildings: 
        count+=1
        clear_output(wait=True)
        print(count, row[0])
        
        if isNewBuilding:
            delete_building(row[0])
            last_building=row[0]
                        
        elif row[0] == last_id_deleted:
            isNewBuilding=True

In [ ]:
conn = None
try:
    conn = pg.connect(params)
    cur = conn.cursor()
    cur.execute("SELECT id_lokalId FROM deleted_buildings ORDER BY id_lokalId")
    deleted_buildings = cur.fetchall() 
    conn.commit()
    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

In [ ]:
delete_values(deleted_buildings,last_id_deleted='')

### Fixing buildings with issues in footprint and floor area


Some buildings have incorrect values reported for the footprint or the floor area, but one of these values is still correctly entered. In such cases, we try to approximate the wrong value based on the correct value. As above, we will first copy original records into a separate table, then modify the records in the main table.

In [ ]:
SQLmodified="""
CREATE TABLE IF NOT EXISTS public.modified_buildings
(
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    jordstykke integer,
    byg007Bygningsnummer integer,
    grund character varying(50) COLLATE pg_catalog."default",
    husnummer character varying(50) COLLATE pg_catalog."default",
    byg404Koordinat character varying(40) COLLATE pg_catalog."default",
    byg026Opførelsesår smallint,
    byg027OmTilbygningsår smallint,
    byg021BygningensAnvendelse character varying(30) COLLATE pg_catalog."default",
    byg041BebyggetAreal integer,
    byg038SamletBygningsareal integer,
    byg040BygningensSamledeErhvervsAreal integer,
    byg039BygningensSamledeBoligAreal integer,
    byg042ArealIndbyggetGarage integer,
    byg043ArealIndbyggetCarport integer,
    byg044ArealIndbyggetUdhus integer,
    byg045ArealIndbyggetUdestueEllerLign integer,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen integer,
    byg047ArealAfAffaldsrumITerrænniveau integer,
    byg048AndetAreal integer,
    byg049ArealAfOverdækketAreal integer,
    byg050ArealÅbneOverdækningerPåBygningenSamlet integer,
    byg051Adgangsareal integer,
    byg054AntalEtager smallint,
    byg055AfvigendeEtager smallint,
    byg056Varmeinstallation smallint,
    byg057Opvarmningsmiddel smallint,
    byg058SupplerendeVarme smallint,
    byg032YdervæggensMateriale smallint,
    byg034SupplerendeYdervæggensMateriale smallint,
    byg033Tagdækningsmateriale smallint,
    byg035SupplerendeTagdækningsMateriale smallint,
    byg036AsbestholdigtMateriale smallint,
    byg130ArealAfUdvendigEfterisolering integer,
    byg071BevaringsværdighedReference smallint,
    byg150Gulvbelægning character varying(50) COLLATE pg_catalog."default",
    byg151Frihøjde character varying(50) COLLATE pg_catalog."default",
    roof_pitch smallint,
    int_wall_surface_lb real,
    int_wall_surface_nlb real,
    CONSTRAINT modified_buildings_pkey PRIMARY KEY (id_lokalId)
);
"""

In [ ]:
run_sql(params,SQLmodified)

In [ ]:
SQLcopy2="""
INSERT INTO modified_buildings 
(
    id_lokalId,
    kommunekode,
    jordstykke,
    byg007Bygningsnummer,
    grund,
    husnummer,
    byg404Koordinat,
    byg026Opførelsesår,
    byg027OmTilbygningsår,
    byg021BygningensAnvendelse,
    byg041BebyggetAreal,
    byg038SamletBygningsareal,
    byg040BygningensSamledeErhvervsAreal,
    byg039BygningensSamledeBoligAreal,
    byg042ArealIndbyggetGarage,
    byg043ArealIndbyggetCarport,
    byg044ArealIndbyggetUdhus,
    byg045ArealIndbyggetUdestueEllerLign,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen,
    byg047ArealAfAffaldsrumITerrænniveau,
    byg048AndetAreal,
    byg049ArealAfOverdækketAreal,
    byg050ArealÅbneOverdækningerPåBygningenSamlet,
    byg051Adgangsareal,
    byg054AntalEtager,
    byg055AfvigendeEtager,
    byg056Varmeinstallation,
    byg057Opvarmningsmiddel,
    byg058SupplerendeVarme,
    byg032YdervæggensMateriale,
    byg034SupplerendeYdervæggensMateriale,
    byg033Tagdækningsmateriale,
    byg035SupplerendeTagdækningsMateriale,
    byg036AsbestholdigtMateriale,
    byg130ArealAfUdvendigEfterisolering,
    byg071BevaringsværdighedReference,
    byg150Gulvbelægning,
    byg151Frihøjde,
    roof_pitch,
    int_wall_surface_lb,
    int_wall_surface_nlb
)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
ON CONFLICT ON CONSTRAINT modified_buildings_pkey DO NOTHING"""

We modify buildings where the footprint or floor area are missing, or either value is equal or lower to 1 (for some buildings, users have entered the value 1 as a placeholder), or where the footprint is higher than the floor area.

In [ ]:
conn = None
try:
    conn = pg.connect(params, row_factory=pg.rows.dict_row)
    cur = conn.cursor()
    cur.execute("DELETE FROM modified_buildings")
    cur.execute("SELECT * FROM buildings ORDER BY id_lokalId")
    row = cur.fetchone() # Get properties from the first building as a dictionary
    print(row)
    
    row_number=0

    while row is not None:
        row_number+=1
        clear_output(wait=True)
        bbrid=row['id_lokalId'.lower()]
        builtarea=row['byg041BebyggetAreal'.lower()]
        floorarea=row['byg038SamletBygningsareal'.lower()]
        nfloors=row['byg054AntalEtager'.lower()]
        print(row_number,bbrid,builtarea,floorarea,nfloors)
        if builtarea is None or builtarea <=1 or floorarea is None or floorarea <=1 or builtarea > floorarea:
            cur_copy=conn.cursor()
            cur_copy.execute(SQLcopy2, tuple(row.values()))
            conn.commit()
            cur_copy.close()
        
        row=cur.fetchone() # Retrieve the next building as a tuple and iterate
    
    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

When the footprint is wrong, we set the footprint equal to the floor area divided by number of floors.

In [ ]:
SQLfootprint="""UPDATE buildings
SET byg041BebyggetAreal = (CASE WHEN (mb.byg054AntalEtager IS NOT NULL AND mb.byg054AntalEtager >0) THEN mb.byg038SamletBygningsareal/mb.byg054AntalEtager ELSE mb.byg038SamletBygningsareal END)
FROM modified_buildings mb 
WHERE ((mb.byg041BebyggetAreal IS NULL OR mb.byg041BebyggetAreal <=1) AND buildings.id_lokalid=mb.id_lokalid)"""

When the floor area is wrong, we set the floor area equal to the footprint times the number of floors.

In [ ]:
SQLfloor="""UPDATE buildings
SET byg038SamletBygningsareal = (CASE WHEN (mb.byg054AntalEtager IS NOT NULL AND mb.byg054AntalEtager >0) THEN mb.byg041BebyggetAreal*mb.byg054AntalEtager ELSE mb.byg041BebyggetAreal END)
FROM modified_buildings mb 
WHERE ((mb.byg038SamletBygningsareal IS NULL OR mb.byg038SamletBygningsareal <=1) AND buildings.id_lokalid=mb.id_lokalid)"""

When the footprint is higher than the floor area, we set the floor area equal to the footprint.

In [ ]:
SQLsup="""UPDATE buildings
SET byg038SamletBygningsareal = mb.byg041BebyggetAreal
FROM modified_buildings mb 
WHERE mb.byg041BebyggetAreal > mb.byg038SamletBygningsareal AND buildings.id_lokalid=mb.id_lokalid"""

In [ ]:
run_sql(params,SQLfootprint)

In [ ]:
run_sql(params,SQLfloor)

In [ ]:
run_sql(params,SQLsup)

### Finding duplicates

Some records have different IDs, but almost all other fields are identical, which could indicate that they represent the same building.

In [ ]:
SQLduplicate="""
CREATE TABLE IF NOT EXISTS public.duplicate_buildings
(
    id_lokalId character varying(50) COLLATE pg_catalog."default" NOT NULL,
    kommunekode smallint,
    jordstykke integer,
    byg007Bygningsnummer integer,
    grund character varying(50) COLLATE pg_catalog."default",
    husnummer character varying(50) COLLATE pg_catalog."default",
    byg404Koordinat character varying(40) COLLATE pg_catalog."default",
    byg026Opførelsesår smallint,
    byg027OmTilbygningsår smallint,
    byg021BygningensAnvendelse character varying(30) COLLATE pg_catalog."default",
    byg041BebyggetAreal integer,
    byg038SamletBygningsareal integer,
    byg040BygningensSamledeErhvervsAreal integer,
    byg039BygningensSamledeBoligAreal integer,
    byg042ArealIndbyggetGarage integer,
    byg043ArealIndbyggetCarport integer,
    byg044ArealIndbyggetUdhus integer,
    byg045ArealIndbyggetUdestueEllerLign integer,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen integer,
    byg047ArealAfAffaldsrumITerrænniveau integer,
    byg048AndetAreal integer,
    byg049ArealAfOverdækketAreal integer,
    byg050ArealÅbneOverdækningerPåBygningenSamlet integer,
    byg051Adgangsareal integer,
    byg054AntalEtager smallint,
    byg055AfvigendeEtager smallint,
    byg056Varmeinstallation smallint,
    byg057Opvarmningsmiddel smallint,
    byg058SupplerendeVarme smallint,
    byg032YdervæggensMateriale smallint,
    byg034SupplerendeYdervæggensMateriale smallint,
    byg033Tagdækningsmateriale smallint,
    byg035SupplerendeTagdækningsMateriale smallint,
    byg036AsbestholdigtMateriale smallint,
    byg130ArealAfUdvendigEfterisolering integer,
    byg071BevaringsværdighedReference smallint,
    byg150Gulvbelægning character varying(50) COLLATE pg_catalog."default",
    byg151Frihøjde character varying(50) COLLATE pg_catalog."default",
    roof_pitch smallint,
    int_wall_surface_lb real,
    int_wall_surface_nlb real,
    CONSTRAINT duplicate_buildings_pkey PRIMARY KEY (id_lokalId)
);
"""

In [ ]:
run_sql(params,SQLduplicate)

In [ ]:
SQLcopy3="""
INSERT INTO duplicate_buildings 
(
    id_lokalId,
    kommunekode,
    jordstykke,
    byg007Bygningsnummer,
    grund,
    husnummer,
    byg404Koordinat,
    byg026Opførelsesår,
    byg027OmTilbygningsår,
    byg021BygningensAnvendelse,
    byg041BebyggetAreal,
    byg038SamletBygningsareal,
    byg040BygningensSamledeErhvervsAreal,
    byg039BygningensSamledeBoligAreal,
    byg042ArealIndbyggetGarage,
    byg043ArealIndbyggetCarport,
    byg044ArealIndbyggetUdhus,
    byg045ArealIndbyggetUdestueEllerLign,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen,
    byg047ArealAfAffaldsrumITerrænniveau,
    byg048AndetAreal,
    byg049ArealAfOverdækketAreal,
    byg050ArealÅbneOverdækningerPåBygningenSamlet,
    byg051Adgangsareal,
    byg054AntalEtager,
    byg055AfvigendeEtager,
    byg056Varmeinstallation,
    byg057Opvarmningsmiddel,
    byg058SupplerendeVarme,
    byg032YdervæggensMateriale,
    byg034SupplerendeYdervæggensMateriale,
    byg033Tagdækningsmateriale,
    byg035SupplerendeTagdækningsMateriale,
    byg036AsbestholdigtMateriale,
    byg130ArealAfUdvendigEfterisolering,
    byg071BevaringsværdighedReference,
    byg150Gulvbelægning,
    byg151Frihøjde,
    roof_pitch,
    int_wall_surface_lb,
    int_wall_surface_nlb
)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
ON CONFLICT ON CONSTRAINT duplicate_buildings_pkey DO NOTHING
"""

First, we select all buildings that share similar values in *kommunekode, byg404koordinat, byg007bygningsnummer, grund, jordstykke, husnummer, byg021bygningensanvendelse, byg041bebyggetareal, byg038samletbygningsareal* and *byg026opførelsesår* with at least one other building.

In [ ]:
conn = None
try:
    conn = pg.connect(params)
    cur = conn.cursor()
    cur.execute("DELETE FROM duplicate_buildings")
    cur.execute("""WITH t0 AS (SELECT kommunekode, byg404koordinat, byg007bygningsnummer, grund, jordstykke, husnummer, byg021bygningensanvendelse, byg041bebyggetareal, byg038samletbygningsareal, byg026opførelsesår, COUNT(*) nr, ARRAY_AGG(id_lokalid) ids FROM buildings b
            GROUP BY kommunekode, byg404koordinat, byg007bygningsnummer, grund, jordstykke, husnummer, byg021bygningensanvendelse, byg041bebyggetareal, byg038samletbygningsareal, byg026opførelsesår
            HAVING COUNT(*) > 1)
SELECT * FROM buildings b WHERE id_lokalid IN (SELECT UNNEST(ids) FROM t0)
""")
    row = cur.fetchone() 
    print(row)
    
    row_number=0

    while row is not None: 
        cur_copy=conn.cursor()
        cur_copy.execute(SQLcopy3, row)
        conn.commit()
        cur_copy.close()
        
        row=cur.fetchone() # Retrieve the next building as a tuple and iterate
    
    cur.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

Note that we are copying all buildings that have duplicates here (both versions of the duplicate). So we shouldn't delete all buildings from duplicate_buildings, we should keep one of each. This can be implemented as follows:
 1) Selecting one version of each row with *"SELECT DISTINCT"* 
 2) Deleting all rows from *duplicate_buildings* in the *buildings* table, and 
 3) Re-inserting into the *buildings* table the rows from the "SELECT DISTINCT" query.

In [ ]:
conn = None
try:
    conn = pg.connect(params)
    cur = conn.cursor()
    cur.execute("SELECT DISTINCT ON (kommunekode, byg404koordinat, byg007bygningsnummer, grund, jordstykke, husnummer, byg021bygningensanvendelse, byg041bebyggetareal, byg038samletbygningsareal, byg026opførelsesår) * FROM duplicate_buildings")
    reinsert_buildings = cur.fetchall() 
    cur2 = conn.cursor()
    cur2.execute("SELECT id_lokalid FROM duplicate_buildings")
    delete_dupli_buildings = cur2.fetchall() 
    conn.commit()
    cur.close()
    cur2.close()

except (Exception, pg.DatabaseError) as error:
    print('error:', error)
finally:
    if conn is not None:
        conn.close()

In [ ]:
delete_values(delete_dupli_buildings,last_id_deleted='')

In [ ]:
SQLreinsert="""
INSERT INTO buildings 
(
    id_lokalId,
    kommunekode,
    jordstykke,
    byg007Bygningsnummer,
    grund,
    husnummer,
    byg404Koordinat,
    byg026Opførelsesår,
    byg027OmTilbygningsår,
    byg021BygningensAnvendelse,
    byg041BebyggetAreal,
    byg038SamletBygningsareal,
    byg040BygningensSamledeErhvervsAreal,
    byg039BygningensSamledeBoligAreal,
    byg042ArealIndbyggetGarage,
    byg043ArealIndbyggetCarport,
    byg044ArealIndbyggetUdhus,
    byg045ArealIndbyggetUdestueEllerLign,
    byg046SamletArealAfLukkedeOverdækningerPåBygningen,
    byg047ArealAfAffaldsrumITerrænniveau,
    byg048AndetAreal,
    byg049ArealAfOverdækketAreal,
    byg050ArealÅbneOverdækningerPåBygningenSamlet,
    byg051Adgangsareal,
    byg054AntalEtager,
    byg055AfvigendeEtager,
    byg056Varmeinstallation,
    byg057Opvarmningsmiddel,
    byg058SupplerendeVarme,
    byg032YdervæggensMateriale,
    byg034SupplerendeYdervæggensMateriale,
    byg033Tagdækningsmateriale,
    byg035SupplerendeTagdækningsMateriale,
    byg036AsbestholdigtMateriale,
    byg130ArealAfUdvendigEfterisolering,
    byg071BevaringsværdighedReference,
    byg150Gulvbelægning,
    byg151Frihøjde,
    roof_pitch,
    int_wall_surface_lb,
    int_wall_surface_nlb
)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
ON CONFLICT ON CONSTRAINT buildings_pkey DO NOTHING
"""

In [ ]:
for row in reinsert_buildings:
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(params)

        # create a new cursor
        cur = connector.cursor()

        # execute the SQL statement
        cur.execute(SQLreinsert,row)

        # commit the changes to the database
        connector.commit()

        # close communication with the database
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(row, error)

    finally:
        if connector is not None:
            connector.close()